# Test of a Zero-Coupon Bond

We introduce new variants of `marlowe-cli run execute` and `marlowe-cli run withdraw` that automatically handle the UTxOs management, coin selection, and balancing, so that one doesn't need to specify `--tx-in` and `--tx-out` for Marlowe transactions. *These commands are experimental, and lightly tested so far.*

In [1]:
marlowe-cli run auto-execute --help

Usage: marlowe-cli run auto-execute 
         --testnet-magic INTEGER --socket-path SOCKET_FILE 
         [--marlowe-in-file MARLOWE_FILE --tx-in-marlowe TXID#TXIX]
         --marlowe-out-file MARLOWE_FILE --change-address ADDRESS 
         [--required-signer SIGNING_FILE] [--metadata-file METADATA_FILE]
         --out-file FILE [--submit SECONDS] [--print-stats] [--script-invalid]

  [EXPERIMENTAL] Run a Marlowe transaction, selecting transaction inputs and
  outputs automatically.

Available options:
  --testnet-magic INTEGER  Network magic. Defaults to the CARDANO_TESTNET_MAGIC
                           environment variable's value.
  --socket-path SOCKET_FILE
                           Location of the cardano-node socket file. Defaults to
                           the CARDANO_NODE_SOCKET_PATH environment variable's
                           value.
  --marlowe-in-file MARLOWE_FILE
                           JSON file with the Marlowe initial state and initial
                    

In [2]:
marlowe-cli run auto-withdraw --help

Usage: marlowe-cli run auto-withdraw 
         --testnet-magic INTEGER --socket-path SOCKET_FILE
         --marlowe-file MARLOWE_FILE --role-name TOKEN_NAME
         --change-address ADDRESS [--required-signer SIGNING_FILE] 
         [--metadata-file METADATA_FILE] --out-file FILE [--submit SECONDS] 
         [--print-stats] [--script-invalid]

  [EXPERIMENTAL] Withdraw funds from the Marlowe role address, selecting
  transaction inputs and outputs automatically.

Available options:
  --testnet-magic INTEGER  Network magic. Defaults to the CARDANO_TESTNET_MAGIC
                           environment variable's value.
  --socket-path SOCKET_FILE
                           Location of the cardano-node socket file. Defaults to
                           the CARDANO_NODE_SOCKET_PATH environment variable's
                           value.
  --marlowe-file MARLOWE_FILE
                           JSON file with the Marlowe inputs, final state, and
                           final contract.
 

## Preliminaries

Record version numbers

In [3]:
marlowe-cli --version

marlowe-cli 0.0.7.0 @ 25f6808de7aca1ab3d47db52fb4b66cae4228618


In [4]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [5]:
git rev-parse HEAD

25f6808de7aca1ab3d47db52fb4b66cae4228618


Set the location of keys.

In [6]:
TREASURY=treasury

Set the location of the node socket.

In [7]:
export CARDANO_NODE_SOCKET_PATH=node.socket

## Preliminaries

### Setup the faucet.

In [8]:
FAUCET_SKEY_FILE=$TREASURY/payment.skey
FAUCET_ADDRESS=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDRESS"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select Network

In [9]:
export CARDANO_TESTNET_MAGIC=2

Compute the slot-to-time parameters.

In [10]:
SLOT_LENGTH=$(marlowe-cli util slotting --testnet-magic "$CARDANO_TESTNET_MAGIC" --socket-path "$CARDANO_NODE_SOCKET_PATH" | jq .scSlotLength)
SLOT_OFFSET=$(marlowe-cli util slotting --testnet-magic "$CARDANO_TESTNET_MAGIC" --socket-path "$CARDANO_NODE_SOCKET_PATH" | jq .scSlotZeroTime)

### Tip of the Blockchain

In [11]:
TIP=$(cardano-cli query tip --testnet-magic "$CARDANO_TESTNET_MAGIC" | jq '.slot')
NOW="$((TIP*SLOT_LENGTH+SLOT_OFFSET))"
HOUR="$((3600*1000))"

The tip of the blockchain:

In [12]:
echo $TIP

2306189


The current POSIX time implies that the tip of the blockchain should be slightly before the following slot:

In [13]:
echo $(((1000 * $(date -u +%s) - SLOT_OFFSET) / SLOT_LENGTH))

2306204


Tests main fail if this is not the case.

### Participants

#### The Lender

In [14]:
LENDER_PREFIX="$TREASURY/john-fletcher"
LENDER_NAME="John Fletcher"
LENDER_ROLE=JF
LENDER_PAYMENT_SKEY="$LENDER_PREFIX".skey
LENDER_PAYMENT_VKEY="$LENDER_PREFIX".vkey

Create the lender's keys, if necessary.

In [15]:
if [[ ! -e "$LENDER_PAYMENT_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$LENDER_PAYMENT_SKEY"      \
                              --verification-key-file "$LENDER_PAYMENT_VKEY"
fi
LENDER_ADDRESS=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$LENDER_PAYMENT_VKEY")

Address of the lender:

In [16]:
echo "Address: $LENDER_ADDRESS"

Address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the lender's address.

In [17]:
marlowe-cli util faucet --out-file /dev/null                  \
                        --submit 600                          \
                        --lovelace 200000000                  \
                        --faucet-address "$FAUCET_ADDRESS"    \
                        --required-signer "$FAUCET_SKEY_FILE" \
                        "$LENDER_ADDRESS"

TxId "b2d090933347f624e9df5eb8319568003f51ea5672f09c9863a9f5f0ea24324c"


#### The Borrower

In [18]:
BORROWER_PREFIX="$TREASURY/thomas-middleton"
BORROWER_NAME="Thomas Middleton"
BORROWER_ROLE=TM
BORROWER_PAYMENT_SKEY="$BORROWER_PREFIX".skey
BORROWER_PAYMENT_VKEY="$BORROWER_PREFIX".vkey

Create the borrower's keys, if necessary.

In [19]:
if [[ ! -e "$BORROWER_PAYMENT_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$BORROWER_PAYMENT_SKEY"      \
                              --verification-key-file "$BORROWER_PAYMENT_VKEY"
fi
BORROWER_ADDRESS=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$BORROWER_PAYMENT_VKEY")

Address of the borrower:

In [20]:
echo "Address: $BORROWER_ADDRESS"

Address: addr_test1vqetzradrerxgqu6xcuk35qckxkl4hwdz8h82zpld226t8ce30xxn


Fund the borrower's address.

In [21]:
marlowe-cli util faucet --out-file /dev/null                  \
                        --submit 600                          \
                        --lovelace 200000000                  \
                        --faucet-address "$FAUCET_ADDRESS"    \
                        --required-signer "$FAUCET_SKEY_FILE" \
                        "$BORROWER_ADDRESS"

TxId "bb015c2ee20a238d2a00085a131588d755f6afe97f288d9cd5cc43f7fdbcedd9"


### Role Tokens

The lender mints the role tokens.

In [22]:
MINT_EXPIRES=$((TIP + 1000000))
ROLE_CURRENCY=$(
marlowe-cli util mint --required-signer "$LENDER_PAYMENT_SKEY" \
                      --change-address  "$LENDER_ADDRESS"      \
                      --expires "$MINT_EXPIRES"                \
                      --out-file /dev/null                     \
                      --submit=600                             \
                      "$LENDER_ROLE" "$BORROWER_ROLE"          \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                           \
)
LENDER_TOKEN="$ROLE_CURRENCY.$LENDER_ROLE"
BORROWER_TOKEN="$ROLE_CURRENCY.$BORROWER_ROLE"

Here are the role tokens:

In [23]:
echo "Lender token: $LENDER_TOKEN"
echo "Borrower token: $BORROWER_TOKEN"

Lender token: 6b23dc8d7e8cec2d81a919baad7fc9f490959375af285c03275376a5.JF
Borrower token: 6b23dc8d7e8cec2d81a919baad7fc9f490959375af285c03275376a5.TM


Find the transaction output with the borrower's role token.

In [24]:
TX_MINT_BORROWER=$(
marlowe-cli util select --asset-only "$BORROWER_TOKEN"    \
                        "$LENDER_ADDRESS"                 \
| sed -n -e '1{s/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;p}' \
)

Send the borrower their role token.

In [25]:
marlowe-cli transaction simple --tx-in "$TX_MINT_BORROWER"                            \
                               --tx-out "$BORROWER_ADDRESS+2000000+1 $BORROWER_TOKEN" \
                               --required-signer "$LENDER_PAYMENT_SKEY"               \
                               --change-address "$LENDER_ADDRESS"                     \
                               --out-file /dev/null                                   \
                               --submit 600

TxId "a35d52283adedd7c4605b5fdbac22fffa1845f8a9c2a9959398e3cc2ffa13467"


## The Contract

The parameters for the contract:

In [26]:
MINIMUM_ADA=3000000
PRINCIPAL=100000000
INTEREST=5000000
LENDING_DEADLINE=$((NOW+12*HOUR))
REPAYMENT_DEADLINE=$((NOW+24*HOUR))

The contract has a minimum-ADA requirement and two timeouts.

In [27]:
echo "Lending deadline: $(date -u -R -d @$((LENDING_DEADLINE/1000)))"
echo "Repayment deadline: $(date -u -R -d @$((REPAYMENT_DEADLINE/1000)))"

Lending deadline: Mon, 05 Sep 2022 04:36:29 +0000
Repayment deadline: Mon, 05 Sep 2022 16:36:29 +0000


We create the contract for the previously specified parameters.

In [28]:
marlowe-cli template zcb --minimum-ada "$MINIMUM_ADA"               \
                         --lender "Role=$LENDER_ROLE"               \
                         --borrower "Role=$BORROWER_ROLE"           \
                         --principal "$PRINCIPAL"                   \
                         --interest "$INTEREST"                     \
                         --lending-deadline "$LENDING_DEADLINE"     \
                         --repayment-deadline "$REPAYMENT_DEADLINE" \
                         --out-contract-file tx-1.contract          \
                         --out-state-file    tx-1.state

## Transaction 1. Create the Contract by Providing the Minimum ADA.

First we create a `.marlowe` file that contains the initial information needed to run the contract. The bare size and cost of the script provide a lower bound on the resources that running it will require.

In [29]:
marlowe-cli run initialize --roles-currency "$ROLE_CURRENCY" \
                           --contract-file tx-1.contract     \
                           --state-file    tx-1.state        \
                           --out-file      tx-1.marlowe      \
                           --print-stats


Validator size: 12412
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18768100, exBudgetMemory = ExMemory 81700}


In particular, we can extract the contract's address from the `.marlowe` file.

In [30]:
CONTRACT_ADDRESS=$(jq -r '.tx.marloweValidator.address' tx-1.marlowe)

The Marlowe contract resides at the following address:

In [31]:
echo $CONTRACT_ADDRESS

addr_test1wqw2qex2sec8e0z2lyn2xkulyzcp7ce2md78v229a5cr4uqx3v466


Because this is a role-based contract, we compute the address of the script for roles.

In [32]:
ROLE_ADDRESS=$(jq -r '.tx.rolesValidator.address' tx-1.marlowe)

The role payout validator resides at the following address:

In [33]:
echo $ROLE_ADDRESS

addr_test1wqfvtv4t6cu059x86qf6kya4ed2ey4rmlf8wvsxug4ahjeskqqgat


The lender submits the transaction along with the minimum Ada required for the contract's initial state. Submitting with the `--print-stats` switch reveals the network fee for the contract, the size of the transaction, and the execution requirements, relative to the protocol limits.

In [34]:
TX_1=$(
marlowe-cli run auto-execute --required-signer "$LENDER_PAYMENT_SKEY" \
                             --marlowe-out-file tx-1.marlowe          \
                             --change-address "$LENDER_ADDRESS"       \
                             --out-file tx-1.raw                      \
                             --print-stats                            \
                             --submit=600                             \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                      \
)


Fee: Lovelace 193529
Size: 521 / 16384 = 3%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%


The contract received the minimum Ada from the first party in the following transaction:

In [35]:
echo $TX_1

2c9cc57c0b11c89843681ffb09010ce957760f819ed443f9a88545b5e99e4c4c


The following UTxOs are at the contract address:

In [36]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2c9cc57c0b11c89843681ffb09010ce957760f819ed443f9a88545b5e99e4c4c     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "520b98d9c584199fad257fb815a9c437b24330a029a11fa6caf16856d1ccbf79"


Here are the UTxOs at the lender's address:

In [37]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2c9cc57c0b11c89843681ffb09010ce957760f819ed443f9a88545b5e99e4c4c     0        176629970 lovelace + TxOutDatumNone
a35d52283adedd7c4605b5fdbac22fffa1845f8a9c2a9959398e3cc2ffa13467     0        7830935 lovelace + TxOutDatumNone
ece363f9f90f3b249cc031dce6c76897dba55d0c7e37a953ada4144d8738d7ca     1        10000000 lovelace + 1 6b23dc8d7e8cec2d81a919baad7fc9f490959375af285c03275376a5.4a46 + TxOutDatumNone


## Transaction 2. Lender Deposits the Loan Amount

First we compute the Marlowe input required to deposit the funds for the loan.

In [38]:
marlowe-cli run prepare --marlowe-file tx-1.marlowe           \
                        --deposit-account "Role=$LENDER_ROLE" \
                        --deposit-party "Role=$LENDER_ROLE"   \
                        --deposit-amount "$PRINCIPAL"         \
                        --invalid-before "$NOW"               \
                        --invalid-hereafter "$((NOW+4*HOUR))" \
                        --out-file tx-2.marlowe               \
                        --print-stats


Datum size: 165
Payment 1
  Acccount: "JF"
  Payee: Party "TM"
  Ada: 100.000000


Now the lender submits the transaction that deposits the loan amount.

In [39]:
TX_2=$(
marlowe-cli run auto-execute --marlowe-in-file tx-1.marlowe           \
                             --tx-in-marlowe "$TX_1#1"                \
                             --required-signer "$LENDER_PAYMENT_SKEY" \
                             --marlowe-out-file tx-2.marlowe          \
                             --change-address "$LENDER_ADDRESS"       \
                             --out-file tx-2.raw                      \
                             --print-stats                            \
                             --submit=600                             \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                      \
)


Fee: Lovelace 1320987
Size: 13389 / 16384 = 81%
Execution units:
  Memory: 7274318 / 14000000 = 51%
  Steps: 1900207221 / 10000000000 = 19%


The contract passed the deposit from the lender to the role address in the following transaction:

In [40]:
echo $TX_2

7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929


The following UTxOs are at the contract address:

In [41]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "ccec310e78cfc9849b58b2946b8f2b9b59dae58ef165173240166aeb30bdfdc9"


Here are the UTxOs at the lender's address:

In [42]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929     0        84291823 lovelace + TxOutDatumNone
7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929     3        1017160 lovelace + 1 6b23dc8d7e8cec2d81a919baad7fc9f490959375af285c03275376a5.4a46 + TxOutDatumNone
a35d52283adedd7c4605b5fdbac22fffa1845f8a9c2a9959398e3cc2ffa13467     0        7830935 lovelace + TxOutDatumNone


Here is the UTxO at the role address:

In [43]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ROLE_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929     2        100000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "1d6b817dd540b8b4358a4f439ac7eda7a9877fe9f02f6244348d80542cb761b7"


## Transaction 3. Borrower Withdraws Loan.

The borrower submits a transaction to withdraw the loan from the role address.

In [44]:
TX_3=$(
marlowe-cli run auto-withdraw --marlowe-file tx-2.marlowe                \
                              --role-name "$BORROWER_ROLE"               \
                              --required-signer "$BORROWER_PAYMENT_SKEY" \
                              --change-address "$BORROWER_ADDRESS"       \
                              --out-file tx-3.raw                        \
                              --print-stats                              \
                              --submit=600                               \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                         \
)


Fee: Lovelace 406891
Size: 2897 / 16384 = 17%
Execution units:
  Memory: 1340842 / 14000000 = 9%
  Steps: 373353621 / 10000000000 = 3%


The UTxOs at the contract address have not changed:

In [45]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "ccec310e78cfc9849b58b2946b8f2b9b59dae58ef165173240166aeb30bdfdc9"


Here are the UTxOs at the lender's address:

In [46]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929     0        84291823 lovelace + TxOutDatumNone
7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929     3        1017160 lovelace + 1 6b23dc8d7e8cec2d81a919baad7fc9f490959375af285c03275376a5.4a46 + TxOutDatumNone
a35d52283adedd7c4605b5fdbac22fffa1845f8a9c2a9959398e3cc2ffa13467     0        7830935 lovelace + TxOutDatumNone


There is no UTxO at the role address:

In [47]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ROLE_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


## Transaction 4. Borrower Repays the Loan's Principal and Interest

First we compute the Marlowe input required to repay the funds for the loan.

In [48]:
marlowe-cli run prepare --marlowe-file tx-2.marlowe                \
                        --deposit-account "Role=$BORROWER_ROLE"    \
                        --deposit-party "Role=$BORROWER_ROLE"      \
                        --deposit-amount "$((PRINCIPAL+INTEREST))" \
                        --invalid-before "$NOW"                    \
                        --invalid-hereafter "$((NOW+4*HOUR))"      \
                        --out-file tx-4.marlowe                    \
                        --print-stats


Datum size: 25
Payment 1
  Acccount: "TM"
  Payee: Party "JF"
  Ada: 105.000000
Payment 2
  Acccount: "JF"
  Payee: Party "JF"
  Ada: 3.000000


Now the borrower submits a transaction that repays the loan.

In [49]:
TX_4=$(
marlowe-cli run auto-execute --marlowe-in-file tx-2.marlowe             \
                             --tx-in-marlowe "$TX_2#1"                  \
                             --required-signer "$BORROWER_PAYMENT_SKEY" \
                             --marlowe-out-file tx-4.marlowe            \
                             --change-address "$BORROWER_ADDRESS"       \
                             --out-file tx-4.raw                        \
                             --print-stats                              \
                             --submit=600                               \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                        \
)


Fee: Lovelace 1097773
Size: 13066 / 16384 = 79%
Execution units:
  Memory: 4523896 / 14000000 = 32%
  Steps: 1202535149 / 10000000000 = 12%


The closing of the contract where the borrower repaid the loan results in principal plus interest being sent to the lender in the transaction:

In [50]:
echo $TX_4

de898f50fee72850b146dfb14b6dea06b625300d140e7bd52bf4a4fc66349ca5


There is no UTxO at the contract address:

In [51]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The UTxOs at the lender's address have not changed:

In [52]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929     0        84291823 lovelace + TxOutDatumNone
7f176c39444d6c0d4d0f96d2ea55f549eec05f16afa92aef4caf4ccdcd4ff929     3        1017160 lovelace + 1 6b23dc8d7e8cec2d81a919baad7fc9f490959375af285c03275376a5.4a46 + TxOutDatumNone
a35d52283adedd7c4605b5fdbac22fffa1845f8a9c2a9959398e3cc2ffa13467     0        7830935 lovelace + TxOutDatumNone


Here are the UTxOs at the borrower's address:

In [53]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
de898f50fee72850b146dfb14b6dea06b625300d140e7bd52bf4a4fc66349ca5     0        194478176 lovelace + TxOutDatumNone
de898f50fee72850b146dfb14b6dea06b625300d140e7bd52bf4a4fc66349ca5     2        1017160 lovelace + 1 6b23dc8d7e8cec2d81a919baad7fc9f490959375af285c03275376a5.544d + TxOutDatumNone


Here is the UTxO at the role address:

In [54]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ROLE_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
de898f50fee72850b146dfb14b6dea06b625300d140e7bd52bf4a4fc66349ca5     1        108000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "506ad3a3267fc31ec816a97469a7ae6c63f4838f38d2d692fa428fcd219bde29"


## Transaction 5. Lender Withdraws Repayment.

The lender submits a transaction to withdraw the repayment from the role address.

In [55]:
TX_5=$(
marlowe-cli run auto-withdraw --marlowe-file tx-4.marlowe              \
                              --role-name "$LENDER_ROLE"               \
                              --required-signer "$LENDER_PAYMENT_SKEY" \
                              --change-address "$LENDER_ADDRESS"       \
                              --out-file tx-5.raw                      \
                              --print-stats                            \
                              --submit=600                             \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                       \
)


Fee: Lovelace 406891
Size: 2897 / 16384 = 17%
Execution units:
  Memory: 1340842 / 14000000 = 9%
  Steps: 373353621 / 10000000000 = 3%


The UTxOs at the contract address have not changed:

In [56]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$CONTRACT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Here are the UTxOs at the lender's address:

In [57]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a35d52283adedd7c4605b5fdbac22fffa1845f8a9c2a9959398e3cc2ffa13467     0        7830935 lovelace + TxOutDatumNone
f0cfe3b437b845fd55a719bd9ade6931b4e39a10beb050cbe39096688eb122b5     0        84902092 lovelace + TxOutDatumNone
f0cfe3b437b845fd55a719bd9ade6931b4e39a10beb050cbe39096688eb122b5     1        108000000 lovelace + 1 6b23dc8d7e8cec2d81a919baad7fc9f490959375af285c03275376a5.4a46 + TxOutDatumNone


There is no UTxO at the role address:

In [58]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ROLE_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


## Clean Up

In [59]:
marlowe-cli transaction simple --tx-in "$TX_4#0"                                    \
                               --tx-in "$TX_4#2"                                    \
                               --tx-out "$LENDER_ADDRESS+1400000+1 $BORROWER_TOKEN" \
                               --required-signer "$BORROWER_PAYMENT_SKEY"           \
                               --change-address "$FAUCET_ADDRESS"                   \
                               --out-file /dev/null                                 \
                               --submit 600

TxId "f82c3a2a1a17e4f63a832f2cb601a25f79fd2f558b0c5a9c13fd38aae27cd276"


In [60]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [61]:
marlowe-cli util mint --required-signer "$LENDER_PAYMENT_SKEY" \
                      --change-address  "$LENDER_ADDRESS"      \
                      --count -1                               \
                      --expires "$MINT_EXPIRES"                \
                      --out-file /dev/null                     \
                      --submit=600                             \
                      "$LENDER_ROLE" "$BORROWER_ROLE"

PolicyID "6b23dc8d7e8cec2d81a919baad7fc9f490959375af285c03275376a5"


In [62]:
TX=$(
marlowe-cli util select --lovelace-only 1                 \
                        "$LENDER_ADDRESS"                 \
| sed -n -e '1{s/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;p}' \
)

In [63]:
marlowe-cli transaction simple --tx-in "$TX"                            \
                               --required-signer "$LENDER_PAYMENT_SKEY" \
                               --change-address "$FAUCET_ADDRESS"       \
                               --out-file /dev/null                     \
                               --submit 600

TxId "fb2db988b121bcfe3dee95c72337ac9f769a1e14d6502a796425fbca15a294d0"


In [64]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [65]:
marlowe-cli util clean --required-signer "$FAUCET_SKEY_FILE" \
                       --change-address "$FAUCET_ADDRESS"    \
                       --out-file /dev/null                  \
                       --submit=600                          \
> /dev/null
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$FAUCET_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a7740cb73671939b4d5f03c7e965d91341e2fe959919aa93241f15f520f70fe0     0        139745725538 lovelace + TxOutDatumNone
